In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

Sequential 모델은 각 layer에 정확히 하나의 입력 텐서와 하나의 출력 텐서가 있는 일반 레이어 스택에 적합하다.

반대로 다중 입/출력, 레이어 공유, 비선형 토플로지에 적합하지 않다.

In [2]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),  #name: TensorBoard에 주석이 달림
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)

x = tf.ones((3, 3))
y = model(x)

In [3]:
model.layers

In [4]:
model.add(layers.Dense(5, name='layer4'))

In [5]:
model.layers

In [6]:
model.pop()  # layer 제거
print(len(model.layers))

3


## 입력 형상 미리 지정

모델의 입력 형상을 알고 있다면 일반적으로 입력 형상을 지정하는 것이 좋다. 

In [14]:
layer = layers.Dense(3)
layer.weights # layer를 만든 후 처음에는 가중치가 없다. 

[]

가중치는 모양이 입력의 형상에 따라 달라지므로 입력시 가중치를 만든다. 

In [17]:
x = tf.ones((1,4))
y = layer(x)
layer.weights

[<tf.Variable 'dense_2/kernel:0' shape=(4, 3) dtype=float32, numpy=
 array([[-0.56953454, -0.52736485, -0.70265824],
        [ 0.5997859 , -0.49573898, -0.2332564 ],
        [ 0.0050658 ,  0.38958704, -0.86249363],
        [-0.3541718 ,  0.67563415,  0.21605754]], dtype=float32)>,
 <tf.Variable 'dense_2/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

이는 sequential모델에도 적용된다. 

In [18]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)  # No weights at this stage!

x = tf.ones((1, 4))
y = model(x)
print("Number of weights after calling the model:", len(model.weights))  

Number of weights after calling the model: 6


In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (1, 2)                    10        
_________________________________________________________________
dense_2 (Dense)              (1, 3)                    9         
_________________________________________________________________
dense_3 (Dense)              (1, 4)                    16        
Total params: 35
Trainable params: 35
Non-trainable params: 0
_________________________________________________________________


In [13]:
model = keras.Sequential()
model.add(keras.Input(shape=(4,)))
model.add(layers.Dense(2, activation='relu'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.layers

In [16]:
model = keras.Sequential()
model.add(layers.Dense(2, activation='relu', input_shape=(4,)))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


이렇게 미리 input_shape로 입력 모양이 정의된 모델은 항상 가중치가 있고 정의된 출력 모양이 있다. 

## Sequential model을 사용한 특성 추출

Sequential model은 빌드되면 Function API model처럼 동작한다. 이는 모든 layer가 input, output을 갖는다는 것을 의미하는데, 이러한 속성을 사용해서 Sequential 모델 내의 모든 중간 layer들의 출력을 추출하는 모델을 생성할 수 있다. '

In [23]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu", name='intermediate'),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=initial_model.get_layer(name='intermediate').output,
)

x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)